In [1]:
import sagemaker
import boto3
import os
import wandb
from sagemaker.pytorch import PyTorch

In [2]:
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_permissions')['Role']['Arn']

Couldn't call 'get_role' to get Role ARN from role name Shivam to get Role path.


In [3]:
source_dir = os.getcwd()
wandb.sagemaker_auth(path=source_dir)

estimator = PyTorch(entry_point='sage_train.sh',
                    source_dir=source_dir,
                    role=role,
                    py_version='py38',
                    framework_version='1.11.0',
                    instance_count=1,
                    instance_type='ml.g5.2xlarge',
                    use_spot_instances=True,  # Use a spot instance
                    max_run=3*60*60*24,  # Max training time
                    max_wait=3*60*60*24,  # Max training time + spot waiting time seconds
                    hyperparameters={
                        # "encoder": "seresnet18",
                        "n_epochs": 50,
                        "batch_size": 2,
                        "project": "body_cut",
                        "use_wandb": True,
                        "name": "no_windows/",
                    })

In [ ]:
estimator.fit({'train': 's3://ai-sagemaker-datasets/body_reflection_unpaired/'}, wait=True)

2022-05-28 23:18:05 Starting - Starting the training job...
2022-05-28 23:18:07 Starting - Launching requested ML instancesProfilerReport-1653779884: InProgress
.....